<a href="https://colab.research.google.com/github/yannickfunk/EmbeddedMLLab/blob/main/end2end.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys


# if in colab, make colab setup
if 'google.colab' in sys.modules:
    FILENAME = "end2end.ipynb"
    BRANCH = "main"
    !git clone -b $BRANCH https://github.com/yannickfunk/EmbeddedMLLab tmp
    !rm tmp/$FILENAME
    !mv tmp/* .
    !rm -rf tmp
    !rm -rf sample_data
    !pip install -r requirements.txt

In [3]:
from models.tinyyolov2 import TinyYoloV2Original
from utils.dataloader import VOCDataLoaderPerson
import pytorch_lightning as pl

train_loader = VOCDataLoaderPerson(train=True, batch_size=128, shuffle=True)
val_loader = VOCDataLoaderPerson(train=False, batch_size=1)

model = TinyYoloV2Original(num_classes=1)
model.load_pt_from_disk("data/voc_pretrained.pt")
trainer = pl.Trainer(max_epochs=15, accelerator='gpu', log_every_n_steps=10, limit_val_batches=350)

trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)

Using downloaded and verified file: data/VOCtrainval_11-May-2012.tar
Extracting data/VOCtrainval_11-May-2012.tar to data/
Using downloaded and verified file: data/VOCtrainval_11-May-2012.tar
Extracting data/VOCtrainval_11-May-2012.tar to data/


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name  | Type            | Params
-------------------------------------------
0  | loss  | YoloLoss        | 0     
1  | pad   | ReflectionPad2d | 0     
2  | conv1 | Conv2d          | 432   
3  | bn1   | BatchNorm2d     | 32    
4  | conv2 | Conv2d          | 4.6 K 
5  | bn2   | BatchNorm2d     | 64    
6  | conv3 | Conv2d          | 18.4 K
7  | bn3   | BatchNorm2d     | 128   
8  | conv4 | Conv2d          | 73.7 K
9  | bn4   | BatchNorm2d     | 256   
10 | conv5 | Conv2d          | 294 K 
11 | bn5   | BatchNorm2d     | 512   
12 | conv6 | Conv2d          | 1.2 M 
13 | bn6   | BatchNorm2d     | 1.0 K 
14 | conv7 | Conv2d          | 4.7 M 
15 | bn7   | BatchNorm2d     | 2.0 K 
16 | conv8 | Conv2d          | 9.4 M 
17 | bn8   | BatchNorm2d     | 2.0 K 
18 | conv9 | Conv2d          | 30.8 K
-

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f9588bbfa30>
Traceback (most recent call last):
  File "/home/yfunk/anaconda3/envs/embedded-ml/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1466, in __del__
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f9588bbfa30>    
self._shutdown_workers()Traceback (most recent call last):

Exception ignored in:   File "/home/yfunk/anaconda3/envs/embedded-ml/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1466, in __del__
  File "/home/yfunk/anaconda3/envs/embedded-ml/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1449, in _shutdown_workers
<function _MultiProcessingDataLoaderIter.__del__ at 0x7f9588bbfa30>    
    if w.is_alive():self._shutdown_workers()Traceback (most recent call last):


  File "/home/yfunk/anaconda3/envs/embedded-ml/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1466, in __del__
  File "/hom